### To use this notebook, just look through the headers and comments for each method. The final run is at the bottom, make sure to input your quadrant info. Preferably we get 4 of these running at the same time (with 4 different users). Once the quadrant info is ready, just click 'Cell' on the top tool bar and click Run All!

**Make sure to fill out the quadrant data at the very bottom before running.**

## Imports

In [2]:
import urllib.request
import pandas as pd
import pickle
import numpy as np
from ratelimiter import RateLimiter
from lxml import objectify

In [3]:
# examples of arange
np.arange(30.123,30.125,.0001)

array([ 30.123 ,  30.1231,  30.1232,  30.1233,  30.1234,  30.1235,
        30.1236,  30.1237,  30.1238,  30.1239,  30.124 ,  30.1241,
        30.1242,  30.1243,  30.1244,  30.1245,  30.1246,  30.1247,
        30.1248,  30.1249])

## Collect Intersections

In [4]:
# 1250 calls per hour = 30,000 per day, probably will take a few days to get everything at the latest
rate_limit = RateLimiter(max_calls=1249, period=3600)

In [5]:
'''
Parameters
----------
lower_left: tuple of floats (latitude, longitude) for lower left coordinate

upper_right: tuple of floats (latitude, longitude) for lower left coordinate

username: geonames user to use

Returns
-------
df: pandas DataFrame containing the GPS locations of intersections within quadrant (without duplicates is in testing)

'''
def collectQuadrant(lower_left, upper_right, username):
    
    # create grid
    latitudes = np.arange(lower_left[0], upper_right[0], .001)
    longitudes = np.arange(lower_left[1], upper_right[1], .001)
    
    # init data tables
    info_dict = {'Street1': [], 'Street2': [], 'Latitude': [], 'Longitude': []}
    df = None
    
    # go through grid
    for latitude in latitudes:
        
        for longitude in longitudes:
            
            # make sure to stay within rate_limit defined above
            with rate_limit:
                
                info = callAPI(latitude, longitude, username)
                
                if info:
                    
                    info_dict['Street1'].append(info['street1'])
                    info_dict['Street2'].append(info['street2'])
                    info_dict['Latitude'].append(info['lat'])
                    info_dict['Longitude'].append(info['lng'])
                    
        if df is not None:
            
            df = df.append(pd.DataFrame(info_dict, columns = ['Street1', 'Street2', 'Latitude', 'Longitude']))
        
        else:
            
            df = pd.DataFrame(pd.DataFrame(info_dict, columns = ['Street1', 'Street2', 'Latitude', 'Longitude']))
            
            
        
    return df.drop_duplicates()


'''
Parameters
----------
latitude: float, latitude coordinate

longitude: float, longitude coordinate

username: string, geonames user

Returns
-------
dict: keys of ['street1', 'street2', 'lat', 'lng'] containing intersection info

OR 

None: when API messes up 

'''
def callAPI(latitude, longitude, username):
    
    r = urllib.request.urlopen('http://api.geonames.org/findNearestIntersection?lat=' + str(latitude) + '&lng=' + str(longitude) + '&username='+username).read()
    
    try:
        
        root = objectify.fromstring(r)
        return {'street1': str(root.intersection.street1),
                'street2': str(root.intersection.street2),
                'lat': str(root.intersection.lat),
                'lng': str(root.intersection.lng)}
    
    except:
        
        return None
        
    

In [6]:
lower_left = (30.285057, -97.745154) # insert lat,long here
upper_right = (30.287790, -97.741399) # insert lat,long here
user = 'kdavid' # write your geonames info here

df = collectQuadrant(lower_left, upper_right, user)

In [7]:
quadrant_name = 'Quadrant_I.pkl' ### name Quadrant_x.pkl info here (x=I,II,III,IV)
df.to_pickle(quadrant_name) 

In [8]:
df

,Street1,Street2,Latitude,Longitude
0,Rio Grande St,W 22nd St,30.285263,-97.744947
1,W 22nd St,Nueces St,30.285179,-97.743944
2,W 22nd St,San Antonio St,30.285084,-97.742901
3,W 22nd St,Guadalupe St,30.285,-97.74181
4,W 22 1/2 St,Rio Grande St,30.286174,-97.74484
5,Rio Grande St,W 22 1/2 St,30.286174,-97.74484
6,San Antonio St,W 23rd St,30.286561,-97.742754
7,Guadalupe St,W Mall Ut,30.285891,-97.741735
8,W 23rd St,Rio Grande St,30.287233,-97.744742
9,Rio Grande St,W 23rd St,30.287233,-97.744742
